## RNN DEMO

Run the colab Demo :  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Jp5DilVUx9G_sGi-nGhGVFy9OHOCf1XM?usp=sharing)

In [1]:
!pip install -q numpy tensorflow scikit-learn


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## LOAD DATASET

In [3]:

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Download IMDB dataset
print("Loading dataset...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)


Loading dataset...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:

# Get word index mapping
word_index = tf.keras.datasets.imdb.get_word_index()

# Create a reverse mapping to decode the reviews
reverse_word_index = {value: key for key, value in word_index.items()}
decode_review = lambda review: ' '.join([reverse_word_index.get(i - 3, '?') for i in review])



1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Print sample review
print("\nSample review:")
print(decode_review(x_train[0]))
print(f"Label: {'Positive' if y_train[0] == 1 else 'Negative'}")


Sample review:
? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have do

In [6]:
#Parameters
vocab_size = 10000
max_length = 250
embedding_dim = 100
rnn_units = 128
batch_size = 64
epochs = 5

# Pad sequences
print("\nPreparing data...")
x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

# Print data shapes
print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")


Preparing data...
Training data shape: (25000, 250)
Test data shape: (25000, 250)


## BUILD MODEL

In [7]:
# Create validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
print(f"Validation data shape: {x_val.shape}")

# Build the RNN model
print("\nBuilding RNN model...")
model = Sequential([
    Embedding(vocab_size, embedding_dim, mask_zero=True),
    BatchNormalization(),
    SimpleRNN(rnn_units, return_sequences=True,
              kernel_initializer='glorot_uniform',
              recurrent_initializer='orthogonal'),
    BatchNormalization(),
    SimpleRNN(rnn_units//2,
              kernel_initializer='glorot_uniform',
              recurrent_initializer='orthogonal'),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


Validation data shape: (5000, 250)

Building RNN model...


In [8]:
# Compile the model with class weights
total_samples = len(y_train)
pos_samples = np.sum(y_train)
neg_samples = total_samples - pos_samples
class_weight = {
    0: total_samples / (2 * neg_samples),
    1: total_samples / (2 * pos_samples)
}

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Define callbacks
callbacks = [
    ModelCheckpoint(
        'best_rnn_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train the model
print("\nTraining the model...")
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    class_weight=class_weight,
    verbose=1
)



Training the model...
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.5170 - loss: 0.8364
Epoch 1: val_accuracy improved from -inf to 0.50800, saving model to best_rnn_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 140s 417ms/step - accuracy: 0.5171 - loss: 0.8361 - val_accuracy: 0.5080 - val_loss: 0.7059
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step - accuracy: 0.6479 - loss: 0.6279
Epoch 2: val_accuracy improved from 0.50800 to 0.53680, saving model to best_rnn_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 144s 425ms/step - accuracy: 0.6480 - loss: 0.6278 - val_accuracy: 0.5368 - val_loss: 0.8923
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.7402 - loss: 0.5185
Epoch 3: val_accuracy improved from 0.53680 to 0.66780, saving model to best_rnn_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 128s 409ms/step - accuracy: 0.7402 - loss: 0.5185 - val_accuracy: 0.6678 - val_loss: 0.6549
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step - accuracy: 0.6985 - los

In [10]:
# Print training history
print("\nTraining History:")
for epoch in range(len(history.history['loss'])):  # Use actual number of completed epochs
    print(f"Epoch {epoch+1}/{len(history.history['loss'])}:")
    print(f"  Training Loss: {history.history['loss'][epoch]:.4f}")
    print(f"  Training Accuracy: {history.history['accuracy'][epoch]:.4f}")
    print(f"  Validation Loss: {history.history['val_loss'][epoch]:.4f}")
    print(f"  Validation Accuracy: {history.history['val_accuracy'][epoch]:.4f}")



Training History:
Epoch 1/5:
  Training Loss: 0.7611
  Training Accuracy: 0.5244
  Validation Loss: 0.7059
  Validation Accuracy: 0.5080
Epoch 2/5:
  Training Loss: 0.6096
  Training Accuracy: 0.6708
  Validation Loss: 0.8923
  Validation Accuracy: 0.5368
Epoch 3/5:
  Training Loss: 0.5127
  Training Accuracy: 0.7516
  Validation Loss: 0.6549
  Validation Accuracy: 0.6678
Epoch 4/5:
  Training Loss: 0.5573
  Training Accuracy: 0.7152
  Validation Loss: 0.6381
  Validation Accuracy: 0.6656
Epoch 5/5:
  Training Loss: 0.4204
  Training Accuracy: 0.8143
  Validation Loss: 0.6711
  Validation Accuracy: 0.7226


In [11]:
# Evaluate on test set
print("\n" + "="*50)
print("TESTING ON TEST SET")
print("="*50)
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Generate predictions on test set
print("\nGenerating predictions on test set...")
y_pred_prob = model.predict(x_test, verbose=1)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Calculate detailed metrics on test set
print("\nTest Set Classification Report:")
report = classification_report(y_test, y_pred)
print(report)


TESTING ON TEST SET
782/782 ━━━━━━━━━━━━━━━━━━━━ 47s 61ms/step - accuracy: 0.7219 - loss: 0.6621
Test accuracy: 0.7232
Test loss: 0.6594

Generating predictions on test set...
782/782 ━━━━━━━━━━━━━━━━━━━━ 47s 60ms/step

Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.58      0.68     12500
           1       0.67      0.87      0.76     12500

    accuracy                           0.72     25000
   macro avg       0.74      0.72      0.72     25000
weighted avg       0.74      0.72      0.72     25000



In [12]:
# Print confusion matrix for test set
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix (Test Set):")
print(f"                | Predicted Negative | Predicted Positive |")
print(f"Actual Negative | {cm[0][0]:<18} | {cm[0][1]:<18} |")
print(f"Actual Positive | {cm[1][0]:<18} | {cm[1][1]:<18} |")


Confusion Matrix (Test Set):
                | Predicted Negative | Predicted Positive |
Actual Negative | 7188               | 5312               |
Actual Positive | 1608               | 10892              |


In [13]:
# Display sample test set predictions
print("\nSample Test Set Predictions:")
for i in range(5):
    review = decode_review(x_test[i])
    prediction = "Positive" if y_pred[i] == 1 else "Negative"
    actual = "Positive" if y_test[i] == 1 else "Negative"
    confidence = y_pred_prob[i][0]
    print("-" * 80)
    print(f"Review: {review[:100]}...")
    print(f"Predicted: {prediction} (confidence: {confidence:.4f}), Actual: {actual}")

# Calculate additional metrics
accuracy = accuracy_score(y_test, y_pred)
report_dict = classification_report(y_test, y_pred, output_dict=True)
precision = report_dict['weighted avg']['precision']
recall = report_dict['weighted avg']['recall']
f1 = report_dict['weighted avg']['f1-score']


Sample Test Set Predictions:
--------------------------------------------------------------------------------
Review: ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the ...
Predicted: Negative (confidence: 0.0224), Actual: Negative
--------------------------------------------------------------------------------
Review: ? this film requires a lot of patience because it focuses on mood and character development the plot...
Predicted: Positive (confidence: 0.9413), Actual: Positive
--------------------------------------------------------------------------------
Review: ? many animation buffs consider ? ? the great forgotten genius of one special branch of the art pupp...
Predicted: Positive (confidence: 0.6665), Actual: Positive
--------------------------------------------------------------------------------
Review: ? i generally love this type of movie however this time i found myself wanting to kick the screen si...
Predicted: Positive (

In [14]:
# Summary of test results
print("\n" + "="*50)
print("TEST SET SUMMARY")
print("="*50)
print(f"Total test samples: {len(y_test)}")
print(f"Correct predictions: {sum(y_pred == y_test)}")
print(f"Incorrect predictions: {sum(y_pred != y_test)}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Class-specific metrics
print("\nClass-specific metrics:")
print("Negative reviews:")
print(f"  Precision: {report_dict['0']['precision']:.4f}")
print(f"  Recall: {report_dict['0']['recall']:.4f}")
print(f"  F1-Score: {report_dict['0']['f1-score']:.4f}")

print("Positive reviews:")
print(f"  Precision: {report_dict['1']['precision']:.4f}")
print(f"  Recall: {report_dict['1']['recall']:.4f}")
print(f"  F1-Score: {report_dict['1']['f1-score']:.4f}")

print("\nDone!")


TEST SET SUMMARY
Total test samples: 25000
Correct predictions: 18080
Incorrect predictions: 6920
Accuracy: 0.7232
Precision: 0.7447
Recall: 0.7232
F1-Score: 0.7170

Class-specific metrics:
Negative reviews:
  Precision: 0.8172
  Recall: 0.5750
  F1-Score: 0.6751
Positive reviews:
  Precision: 0.6722
  Recall: 0.8714
  F1-Score: 0.7589

Done!
